In [82]:
"""
这一部分用于完成特征向量的测试

ReliefF算法

"""

'\n这一部分用于完成特征向量的测试\n\nReliefF算法\n\n'

In [83]:
import functools
import json
import random

import numpy as np

In [84]:
# 读取标签、训练集以及测试集
with open("categories.json", 'r') as file:
    categories = json.load(file)
with open("train_data_features_2.json", 'r') as file:
    train_data = json.load(file)
with open("test_data_features_2.json", 'r') as file:
    test_data = json.load(file)

In [85]:
# 将训练集转化为Sklearn 能识别的数据集，并且声称相对应的标签
train_data_tmp = None
train_label = None
for i, (key, value) in enumerate(train_data.items()):
    value = np.array(value)
    if train_data_tmp is None:
        train_data_tmp = value.copy()
    else:
        train_data_tmp = np.vstack((train_data_tmp, value))

    if train_label is None:
        train_label = np.full((len(value)), i)
    else:
        train_label = np.hstack((train_label, np.full((len(value)), i)))

train_data = train_data_tmp
print(train_data.shape)
print(train_label.shape)


(600, 25)
(600,)


In [86]:
# ReliefF Algorithm


In [87]:
m=100
q=5

w = np.zeros(25)

def cmps(p1, p2):
    if p1[25] < p2[25]:
        return -1
    else:
        return 1


def diff(A, R1, R2):
   \ return abs(R1[A] - R2[A])
    if abs(R1[A]-R2[A])<0.001:
        return 0
    else:
        return 1


for mi in range(m):
    choose_class = random.randint(0, 600)
    choose_type = choose_class // 60
    choose_feature = train_data[choose_type]

    # 获取同类K个最近邻以及不同类K个最近邻
    k_features=[[],[],[],[],[],[],[],[],[],[]]
    for i in range(600):
        k_features[i // 60].append(np.hstack((train_data[i], np.sqrt(((choose_feature - train_data[i])**2).sum()))))

    # print(len(k_features))
    # print(len(k_features[0]))

    k_same_features = sorted(k_same_features, key=functools.cmp_to_key(cmps))
    for i in range(10):
        k_features[i] = sorted(k_features[i], key=functools.cmp_to_key(cmps))
        k_features[i] = k_features[i][0:q]

    # print(len(k_features))
    # print(len(k_features[0]))

    for a in range(25):
        item1 = 0
        for i in range(q):
            item1 += diff(a, train_data[choose_class], k_features[choose_type][i])
        item1 /= m*q

        item2 = 0
        for j in range(10):
            if j != choose_type:
                item_tmp = 0
                for i in range(q):
                    item_tmp += diff(a, train_data[choose_class], k_features[j][i])
                item_tmp /= m*q
                item2 += item_tmp*(1/9)

        w[a] = w[a] - item1 + item2

print(w)

[ 0.15856915  0.04280697  0.07709978  0.14707888  0.02179735  0.08103083
  0.10399601  0.02295767  0.08733124  0.00752564  0.00477822  0.00101449
  0.00778874  0.00571851  0.00373295  0.00020325  0.00385432  0.00606277
  0.00448655 -0.00071357  0.00340314  0.00581649  0.00374438 -0.00182741
  0.00195079]


In [80]:
# 在此方法下，计算10*25个特征向量
m=20 # 计算次数
q=5 # KNN计算时取的K值

w = np.zeros((10, 25)) # 特征向量


def cmps(p1, p2):
    """
    比较器
    """
    if p1[25] < p2[25]:
        return -1
    else:
        return 1

def diff(A, R1, R2):
    """
    计算样本R1、R2之间某一特征的差异
    """
    return abs(R1[A] - R2[A])

# 主循环
# 10个类别
for choose_type in range(10):
    # 循环次数
    for mi in range(m):
        # 随机从当前类别中选取一个，并获取其特征向量
        choose_class = random.randint(60*choose_type, 60*choose_type+60)
        choose_feature = train_data[choose_type]

        # 计算所有的训练集特征向量与选中的特征向量的距离
        k_features=[[],[],[],[],[],[],[],[],[],[]]
        for i in range(600):
            k_features[i // 60].append(np.hstack((train_data[i], np.sqrt(((choose_feature - train_data[i])**2).sum()))))

        # 排序，并且获取同类K个最近邻以及不同类K个最近邻
        for i in range(10):
            k_features[i] = sorted(k_features[i], key=functools.cmp_to_key(cmps))
            k_features[i] = k_features[i][0:q]

        # 根据公式计算每个特征向量的权值
        for a in range(25):

            # 公式的第二项
            item1 = 0
            for i in range(q):
                item1 += diff(a, choose_feature, k_features[choose_type][i])
            item1 /= m*q

            # 公式的第三项
            item2 = 0
            for j in range(10):
                if j != choose_type:
                    item_tmp = 0
                    for i in range(q):
                        item_tmp += diff(a, choose_feature, k_features[j][i])
                    item_tmp /= m*q
                    item2 += item_tmp*(1/9)
            # 根据公式更新权值
            w[choose_type][a] = w[choose_type][a] - item1 + item2

print(w)


[[ 1.36532191e-01  1.31014090e-01  4.80844900e-02  9.44965761e-02
   2.25696709e-02  6.91875291e-02  6.12709016e-02  7.29392980e-02
   8.17564026e-02  5.24689272e-02  1.63884214e-03  6.72317117e-02
   1.54720064e-01  2.15749210e-02  1.31685748e-03  1.05303966e-01
   2.15194698e-01  5.11131794e-02  1.56046790e-03  7.10719912e-02
   2.06518553e-01  4.38111458e-02  1.32578173e-03  7.27663332e-02
   1.69969121e-01]
 [ 8.81111402e-02  7.58223082e-02  4.67343635e-02  6.45859958e-02
   2.07323865e-02 -1.08368528e-02  2.04658373e-02  4.05231090e-02
   3.71423861e-02  1.17654672e-02  3.33675156e-03  2.11375778e-03
   9.64030139e-03  1.13508967e-02  2.82825388e-03  4.20822264e-03
   1.25038512e-02  1.01247739e-02  3.27763649e-03  8.34680980e-03
   1.60652022e-02  1.50955715e-02  2.85598179e-03  7.95319996e-03
   1.71795772e-02]
 [ 7.05976341e-02  1.59099257e-02  1.24387137e-02  6.80965023e-02
   1.24706162e-02  4.33867358e-02  2.60633803e-03 -2.78056647e-03
   3.65416547e-02 -1.24328549e-03  5.7

In [88]:
with open("weight.json", 'w') as file:
    json.dump(w.tolist(), file)
